In [ ]:
# !pip install yfinance
# !pip install scikit-learn

In [ ]:
###Importing libraries
import pandas as pd
import numpy as np
import yfinance as yf  # We will use this library to upload latest data from Yahoo API
import os, sys
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.model_selection import train_test_split
#currentdir = os.path.dirname(os.path.realpath(__file__))
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
# Importing custom functions
from src.data_collection import get_stock_data
from src.model_building import perform_linear_regression, perform_tuned_regression,train_xgboost_model_tuned,train_xgboost_model
from src.data_preprocessing import preprocess_stock_data
from src.feature_engineering import cal_ex_moving_avg, calculate_rsi
from src.model_evaluation import evaluate_model
from src.utils import save_model,save_raw_data,save_processed_data

In [ ]:
# Function to get stock data
def get_stock_df(stock_name, period):
    """This functions helps in getting the stock data 
    from yahoo finance API"""
    stock_df = get_stock_data(stock_name, period)
    return stock_df

In [ ]:
# Function to calculate momentum and plot buy/sell signals
def calculate_momentum(stock_data):
    """
    Calculate the momentum of stock data.

    Parameters:
    - stock_data (DataFrame): DataFrame containing stock data with a 'Close' column.

    Returns:
    - DataFrame: Original DataFrame with an added 'momentum' column representing the percentage change in closing prices.
    """
    stock_data['momentum'] = stock_data['Close'].pct_change()
    return stock_data

In [ ]:
def calculate_features(stock_data):
    """
    Calculate Exponential Moving Average (EMA) and Relative Strength Index (RSI) for stock data.

    Parameters:
    - stock_data (DataFrame): DataFrame containing stock data.

    Returns:
    - DataFrame: Original DataFrame with additional columns for EMA and RSI.
    """
    stock_data = cal_ex_moving_avg(stock_data, EMA_window=3)
    stock_data = calculate_rsi(stock_data, window_days=14)
    return stock_data


In [ ]:
def split_data(stock_data):
    """
    Split stock data into training and testing sets.

    Parameters:
    - stock_data (DataFrame): DataFrame containing stock data with columns 'Open', 'High', 'Low', 'Volume', 'EMA', 'RSI', and 'Close'.

    Returns:
    - tuple: A tuple containing four elements - train_x (training features), test_x (testing features), train_y (training target), test_y (testing target).
    """
    x = stock_data[['Open', 'High', 'Low', 'Volume', 'EMA', 'RSI']]
    y = stock_data['Close']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.15, shuffle=False, random_state=0)
    return train_x, test_x, train_y, test_y

In [ ]:
def df_plot(data, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    """
    Plot visualizations of stock data.

    Parameters:
    - data (DataFrame): DataFrame containing stock data.
    - x (array-like): X-axis data, typically representing dates.
    - y (array-like): Y-axis data, corresponding to the values to be plotted.
    - title (str): Title of the plot (default is an empty string).
    - xlabel (str): Label for the X-axis (default is 'Date').
    - ylabel (str): Label for the Y-axis (default is 'Value').
    - dpi (int): Dots per inch for the resolution of the plot (default is 100).

    Returns:
    - None
    """
    plt.figure(figsize=(10, 3), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

In [ ]:
stock_name = "NIO"
period = "2mo"

# Step 1: Get stock data
stock_data = get_stock_df(stock_name, period)
print(stock_data.shape)
stock_data.head(5)

In [ ]:
data_dir = "D:/Python/Stock_price_prediction/Stock_price_prediction/data/raw"

In [ ]:
save_raw_data(stock_data,stock_name,data_dir)

In [ ]:
# Step 2: Preprocess stock data
stock_data = preprocess_stock_data(stock_data)

In [ ]:
# Step 3: Plot stock data
df_plot(stock_data, stock_data.index, stock_data['Close'], title=(stock_name, "History stock performance till date"))

In [ ]:
# Step 4: Calculate momentum and plot buy/sell signals
stock_data = calculate_momentum(stock_data)
df_plot(stock_data, stock_data.index, stock_data['momentum'], title=(stock_name, "Momentum Changes"))

In [ ]:
# Step 5: Calculate EMA and RSI
stock_data = calculate_features(stock_data)
stock_data.head(5)

In [ ]:
data_dir = "D:/Python/Stock_price_prediction/Stock_price_prediction/data/processed"
save_processed_data(stock_data,stock_name,data_dir)

In [ ]:
# Step 6: Split data into training and testing sets
train_x, test_x, train_y, test_y = split_data(stock_data)

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
# Step 7: Perform linear regression without hyperparameter tuning
model_before_tuning, X_train, X_test, y_train, y_test = perform_linear_regression(train_x, test_x, train_y,test_y)

# Make predictions
predicted_prices_before_tuning = model_before_tuning.predict(X_test)

# Evaluate model
evaluation_metrics_before_tuning = evaluate_model(y_test, predicted_prices_before_tuning)

In [ ]:
# Step 8: Evaluate results before tuning
print("Results before hyperparameter tuning:")
print("Evaluation metrics before tuning:")
print("Confidence (R-squared) before tuning:", evaluation_metrics_before_tuning["R-squared Score"])
print("Mean Squared Error before tuning:", evaluation_metrics_before_tuning["Mean Squared Error"])
print("Mean Absolute Percentage Error (MAPE) before tuning:", evaluation_metrics_before_tuning["Mean Absolute Percentage Error (MAPE)"])


In [ ]:
# Step 9: Perform linear regression with hyperparameter tuning
model_after_tuning, confidence_after_tuning_lr, predicted_prices_after_tuning, evaluation_metrics_after_tuning = perform_tuned_regression(train_x, test_x, train_y, test_y)

# Step 10: Evaluate results after tuning
print("\nResults after hyperparameter tuning:")
print("Confidence (R-squared) after tuning:", confidence_after_tuning_lr)
print("Mean Squared Error after tuning:", evaluation_metrics_after_tuning["Mean Squared Error"])
print("Mean Absolute Percentage Error (MAPE) after tuning:", evaluation_metrics_after_tuning["Mean Absolute Percentage Error (MAPE)"])



In [ ]:
# Train the XGBoost model
trained_model, X_train_used, y_train_used, X_test_used, y_test_used = train_xgboost_model(X_train, y_train, X_test, y_test)

# Predict the test data
y_pred = trained_model.predict(X_test_used)

# Evaluate the trained model using your evaluate_model function
evaluation_metrics_before_tuning_xgb = evaluate_model(y_test_used, y_pred)

# Print or use the evaluation results as needed
print("XGBoost Results before hyperparameter tuning:")
print("Evaluation metrics before tuning:")
print("Confidence (R-squared) before tuning:", evaluation_metrics_before_tuning_xgb["R-squared Score"])
print("Mean Squared Error before tuning:", evaluation_metrics_before_tuning_xgb["Mean Squared Error"])
print("Mean Absolute Percentage Error (MAPE) before tuning:", evaluation_metrics_before_tuning_xgb["Mean Absolute Percentage Error (MAPE)"])


In [ ]:
### Define the hyperparameters grid for tuning
params = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 500, 1000],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    # 'reg_alpha': [0, 0.1, 0.5, 1],  # L1 regularization
    # 'reg_lambda': [0, 0.1, 0.5, 1] 
}

In [ ]:
###Training XGBOOST with hyperparameters
res_tuple = train_xgboost_model_tuned(X_train, y_train, X_test, y_test, params)
best_model, best_params = res_tuple[:2]  # Extract the first two elements

In [ ]:
####predicting on X test data
y_pred = best_model.predict(X_test)

In [ ]:
###evaluate the best model separately using your evaluate_model function
evaluation_metrics_after_tuning_xgb = evaluate_model( y_test,y_pred)

# Print or use the evaluation metrics as needed
print("Evaluation Metrics:")
print("\nResults after hyperparameter tuning:")
print("Confidence (R-squared) after tuning:", evaluation_metrics_after_tuning_xgb['R-squared Score'])
print("Mean Squared Error after tuning:", evaluation_metrics_after_tuning_xgb["Mean Squared Error"])
print("Mean Absolute Percentage Error (MAPE) after tuning:", evaluation_metrics_after_tuning_xgb["Mean Absolute Percentage Error (MAPE)"])

In [ ]:
# Define the results and model names
model_names = ['Linear Before Tuning', 'Linear After Tuning', 'XGBoost Before Tuning', 'XGBoost After Tuning']
mse_values = [evaluation_metrics_before_tuning["Mean Squared Error"], evaluation_metrics_after_tuning["Mean Squared Error"],
              evaluation_metrics_before_tuning_xgb["Mean Squared Error"], evaluation_metrics_after_tuning_xgb["Mean Squared Error"]]
confidence_values = [evaluation_metrics_before_tuning['R-squared Score'], evaluation_metrics_after_tuning['R-squared Score'],evaluation_metrics_before_tuning_xgb["R-squared Score"] ,evaluation_metrics_after_tuning_xgb['R-squared Score'] ]  # Fill in with actual confidence values
mape_values = [evaluation_metrics_before_tuning["Mean Absolute Percentage Error (MAPE)"], evaluation_metrics_after_tuning["Mean Absolute Percentage Error (MAPE)"],
               evaluation_metrics_before_tuning_xgb["Mean Absolute Percentage Error (MAPE)"], evaluation_metrics_after_tuning_xgb["Mean Absolute Percentage Error (MAPE)"]]

# Create DataFrame
results_df = pd.DataFrame({
    'Model Name': model_names,
    'MSE': mse_values,
    'Confidence (R-squared)': confidence_values,
    'MAPE': mape_values
})

# Print the DataFrame
print(results_df)

In [ ]:
# Calculate the baseline MSE based on the minimum MSE among the models
baseline_mse = results_df['MSE'].min()

# Compare results and save the best model if it meets the criteria
best_model_name = results_df.loc[results_df['MSE'].idxmin(), 'Model Name']
best_model_mse = results_df.loc[results_df['MSE'].idxmin(), 'MSE']

# Get the best model object based on the best_model_name
if best_model_name == 'Linear Before Tuning':
    best_model = perform_linear_regression
elif best_model_name == 'Linear After Tuning':
    best_model = perform_tuned_regression
elif best_model_name == 'XGBoost Before Tuning':
    best_model = train_xgboost_model
elif best_model_name == 'XGBoost After Tuning':
    best_model = train_xgboost_model_tuned
else:
    best_model = None

# Save the best model if it's not None
if best_model is not None:
    model_dir = 'D:\\Python\\Stock_price_prediction\\Stock_price_prediction\\models'
    save_model(best_model,stock_name,best_model_name,model_dir)
    print("Best model saved successfully.")
else:
    print("No improvement in model performance. Best model not saved.")

